# ETL Script for TFM ITI

### Import libraries

In [ ]:
pip install elasticsearch

In [3]:
from elasticsearch import Elasticsearch
from io import StringIO
import pandas as pd

In [1]:
import requests
import json
import pprint

### Import docs from Elasticsearch 

In [4]:
es = Elasticsearch('http://elasticsearch:9200')

In [11]:
size = 10000
res = es.search(index="logs-raw", body={"query": {"match_all": {}}}, size = size)

> Size = 10000 is the max number of hits that a search can get

### Create Dataframe 

In [19]:
#Create empty DataFrame
raw_df = pd.DataFrame(columns = ["index","method","url","protocol","userAgent","pragma","cacheControl","accept","acceptEncoding","acceptCharset","acceptLanguage","host","connection","contentLength","contentType","cookie","payload","label"])

In [20]:
#Add every single log to the DataFrame
for i in range(size):
    message = res["hits"]["hits"][i]["_source"]["message"]
    data = StringIO(message)
    df = pd.read_csv(data, sep = ",", names = ["index","method","url","protocol","userAgent","pragma","cacheControl","accept","acceptEncoding","acceptCharset","acceptLanguage","host","connection","contentLength","contentType","cookie","payload","label"])
    raw_df = raw_df.append(df, ignore_index=True)

In [ ]:
result = raw_df.to_json(orient="index")
document = json.loads(result) 

### Store clean DataFrame in different Elasticsearch index

In [60]:
requests.put('http://elasticsearch:9200/logs-clean')

for i in range(size):
    requests.post('http://elasticsearch:9200/logs-clean/doc/',json=document[str(i)])